# Need minimum L4 GPU

In [ ]:
# @title 1. Set up environment

# Clone the repository
print("Cloning BagelUI fork repository...")
!git clone https://github.com/dasjoms/BagelUI.git

# Navigate into the BAGEL directory
%cd BagelUI

# --- Dependency Installation ---
# Colab typically has some versions of torch/torchvision pre-installed.
print("\nUninstalling pre-existing torch/torchvision...")
!pip uninstall torch torchvision torchaudio -y

print("\nInstalling pip requirements from requirements.txt...")
# Install dependencies from requirements.txt
!pip install -r requirements.txt

# Force numpy upgrade
!pip install "numpy>=1.25.0,<2.0.0" --upgrade

print("\nInstalling flash_attn==2.5.8...")
# Install flash_attn. (This may take a while)
!pip install flash_attn==2.5.8 --no-build-isolation -vvv

print("\nAll dependencies installed!")

Cloning BAGEL repository...
fatal: destination path 'BagelUI' already exists and is not an empty directory.
/content/BagelUI

Uninstalling pre-existing torch/torchvision...

Installing pip requirements from requirements.txt...
Ignoring triton-windows: markers 'sys_platform == "win32"' don't match your environment
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 125.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.0/468.0 kB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 116.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 435

  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.4
    Uninstalling numpy-1.24.4:
      Successfully uninstalled numpy-1.24.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 25.2.1 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 11.0.0 which is incompatible.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.10.1 which is incompatible.
dask-expr 1.1.21 requires pyarrow>=14.0.1, but you have pyarrow 11.0.0 which is incompatible.
plotnine 0.14.5 requires matplotlib>=3.8.0, but you have matplotlib 3.7.0 which is incompatible.
pylibcudf-cu12 25.2.1 requires pyarr

In [ ]:
# @title 2. Download pretrained checkpoint
from huggingface_hub import snapshot_download
import os

print("Starting model download...")

os.makedirs("models", exist_ok=True)

save_dir = "models/BAGEL-7B-MoT"
repo_id = "ByteDance-Seed/BAGEL-7B-MoT"
cache_dir = os.path.join(save_dir, "cache")

snapshot_download(cache_dir=cache_dir,
  local_dir=save_dir,
  repo_id=repo_id,
  local_dir_use_symlinks=False,
  resume_download=True,
  allow_patterns=["*.json", "*.safetensors", "*.bin", "*.py", "*.md", "*.txt"],
)

print(f"\nModel downloaded to: {save_dir}")

In [ ]:
# @title 3. Use Gradio WebUI to start playing with BAGEL!
# Comment out '--mode 2' if using A100 GPU

!python app.py --share --mode 2

2025-06-02 21:10:41.926745: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-02 21:10:41.951495: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748898641.974016  191542 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748898641.981145  191542 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-02 21:10:42.006322: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr